In [ ]:
!pip install datasets
from datasets import load_dataset
dataset = load_dataset("ag_news")

count0, count1, count2, count3 = 0, 0, 0, 0
dataset_short = []
for i in range(len(dataset['train'])):
  if dataset['train'][i]['label'] == 0 and count0 < 2000:
    dataset_short.append({'news': dataset['train'][i]['text'], 'label': 0})
    count0 += 1
  elif dataset['train'][i]['label'] == 1 and count1 < 2000:
    dataset_short.append({'news': dataset['train'][i]['text'], 'label': 1})
    count1 += 1
  elif dataset['train'][i]['label'] == 2 and count2 < 2000:
    dataset_short.append({'news': dataset['train'][i]['text'], 'label': 2})
    count2 += 1
  elif dataset['train'][i]['label'] == 3 and count3 < 2000:
    dataset_short.append({'news': dataset['train'][i]['text'], 'label': 3})
    count3 += 1
len(dataset_short)

In [ ]:
import random
random.shuffle(dataset_short)
dataset = {'train': dataset_short[:6400], 'test': dataset_short[6400:]}

# Выводим объём выборок
print('Объём train:', len(dataset['train']))
print('Объём test:', len(dataset['test']))

train_texts_news = [s['news'] for s in dataset['train']]
test_texts_news = [s['news'] for s in dataset['test']]

train_labels_news = [s['label'] for s in dataset['train']]
test_labels_news = [s['label'] for s in dataset['test']]

# ВЫводим примеры
print(train_texts_news[0], train_labels_news[0])
print(test_texts_news[0], test_labels_news[0])

In [ ]:
dataset_imdb = load_dataset("imdb")

count0, count1 = 0, 0
dataset_short_imdb = []

for i in range(len(dataset_imdb['train'])):
  if dataset_imdb['train'][i]['label'] == 0 and count0 < 4000:
    dataset_short_imdb.append({'text': dataset_imdb['train'][i]['text'], 'label': 0})
    count0 += 1
  elif dataset_imdb['train'][i]['label'] == 1 and count1 < 4000:
    dataset_short_imdb.append({'text': dataset_imdb['train'][i]['text'], 'label': 1})
    count1 += 1

random.shuffle(dataset_short_imdb)
dataset_imdb = {'train': dataset_short_imdb[:6400], 'test': dataset_short_imdb[6400:]}

# Выводим объём выборок
print('Объём train:', len(dataset_imdb['train']))
print('Объём test:', len(dataset_imdb['test']))

train_texts_imdb = [s['text'] for s in dataset_imdb['train']]
test_texts_imdb = [s['text'] for s in dataset_imdb['test']]

train_labels_imdb = [s['label'] for s in dataset_imdb['train']]
test_labels_imdb = [s['label'] for s in dataset_imdb['test']]

In [ ]:
print(train_texts_imdb[0])
print(train_labels_imdb[0])
print()
print(test_texts_imdb[0])
print(test_labels_imdb[0])

In [ ]:
# news example
news_example = train_texts_news[0]
print(news_example)

# imdb example
imdb_example = train_texts_imdb[0]
print(imdb_example)

# Способы построения словаря (feature space)

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')

import re
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = list(set(stopwords.words("english")))

# stopword - удаляем ли стопслова и спецсимволы? - 0 (нет, оставляем), 1 (удаляем)

def all_words(text, stopword):
  text = text.lower()
  tokens = word_tokenize(text)

  if stopword == 1:
    tokens = [word for word in tokens if word not in stop_words]
    tokens = [re.sub(r'[^\w\s]', '', word) for word in tokens]

  return tokens

In [ ]:
print(all_words(news_example, 0))
print(all_words(imdb_example, 1))

In [ ]:
from nltk import pos_tag
nltk.download('averaged_perceptron_tagger_eng')

def nouns(text, stopword):
  tokens = all_words(text, stopword)
  tagged_words = pos_tag(tokens)
  filter_words = [word for word, tag in tagged_words if tag.startswith("NN")]

  if stopword == 1:
    filter_words = [word for word in filter_words if word not in stop_words]
    filter_words = [re.sub(r'[^\w\s]', '', word) for word in filter_words]

  return filter_words

In [ ]:
print(nouns(news_example, 1))
print(nouns(imdb_example, 0))

In [ ]:
def nouns_adj(text, stopword):
  tokens = all_words(text, stopword)
  tagged_words = pos_tag(tokens)
  filter_words = [word for word, tag in tagged_words if tag.startswith("NN") or tag.startswith("JJ")]

  if stopword == 1:
    filter_words = [word for word in filter_words if word not in stop_words]
    filter_words = [re.sub(r'[^\w\s]', '', word) for word in filter_words]

  return filter_words

In [ ]:
print(nouns_adj(news_example, 0))
print(nouns_adj(imdb_example, 1))

In [ ]:
def nouns_adj_verb(text, stopword):
  tokens = all_words(text, stopword)
  tagged_words = pos_tag(tokens)
  filter_words = [word for word, tag in tagged_words if tag.startswith("NN") or tag.startswith("JJ") or tag.startswith("V")]

  if stopword == 1:
    filter_words = [word for word in filter_words if word not in stop_words]
    filter_words = [re.sub(r'[^\w\s]', '', word) for word in filter_words]

  return filter_words

In [ ]:
print(nouns_adj_verb(news_example, 1))
print(nouns_adj_verb(imdb_example, 0))

# Варианты представления весов признаков

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


# определим методы векторизации
vectorizers = {
    "Binary min": CountVectorizer(binary=True, min_df=10),
    "Binary max": CountVectorizer(binary=True, max_df=0.9),
    "Binary min_max": CountVectorizer(binary=True, min_df=10, max_df=0.9),
    "Binary nothing": CountVectorizer(binary=True),
    "Tf min": CountVectorizer(binary=False, min_df=10),
    "Tf max": CountVectorizer(binary=False, max_df=0.9),
    "Tf min_max": CountVectorizer(binary=False, min_df=10, max_df=0.9),
    "Tf nothing": CountVectorizer(binary=False),
    "TfIdf min": TfidfVectorizer(min_df=10),
    "TfIdf max": TfidfVectorizer(max_df=0.9),
    "TfIdf min_max": TfidfVectorizer(min_df=10, max_df=0.9),
    "TfIdf nothing": TfidfVectorizer()
}

classifier = {
    "Logistic Regression": LogisticRegression(max_iter=2000),
    "Random Forest": RandomForestClassifier(n_estimators=200, random_state=42)
}

In [ ]:
from sklearn.pipeline import Pipeline

def train_and_test(train_texts, test_texts, train_labels, test_labels, dataset_name, feature_space, stopword):
  results = []

  for vec_name, vectorizer in vectorizers.items():
    for clf_name, clf in classifier.items():
      # Создаём pipeline
      pipeline = Pipeline([
        ("vectorizer", vectorizer),
        ("classifier", clf)
      ])

      # Обучаем модель
      pipeline.fit(train_texts, train_labels)

      # Делаем предсказания
      y_pred = pipeline.predict(test_texts)

      # Считаем метрики
      accuracy = accuracy_score(test_labels, y_pred)
      f1 = f1_score(test_labels, y_pred, average="macro" if dataset_name == "news" else "binary")

      # Сохраняем результаты
      results.append([dataset_name, vec_name, clf_name, accuracy, f1, feature_space, stopword])
  return results

In [ ]:
import pandas as pd

itog = []
# теперь проводим эксперименты и сохраняем результаты
for d in ['news', 'imdb']:
  for feature_space in range(1, 5):
    for stopword in [0, 1]:
      if d == 'news':
        train_texts = train_texts_news
        test_texts = test_texts_news
        train_labels = train_labels_news
        test_labels = test_labels_news
      else:
        train_texts = train_texts_imdb
        test_texts = test_texts_imdb
        train_labels = train_labels_imdb
        test_labels = test_labels_imdb

      if feature_space == 1:
        train_texts = [' '.join(all_words(text, stopword)) for text in train_texts]
        test_texts = [' '.join(all_words(text, stopword)) for text in test_texts]
        fs= "all_words"
      elif feature_space == 2:
        train_texts = [' '.join(nouns(text, stopword)) for text in train_texts]
        test_texts = [' '.join(nouns(text, stopword)) for text in test_texts]
        fs = "nouns"
      elif feature_space == 3:
        train_texts = [' '.join(nouns_adj(text, stopword)) for text in train_texts]
        test_texts = [' '.join(nouns_adj(text, stopword)) for text in test_texts]
        fs= "nouns_adj"
      else:
        train_texts = [' '.join(nouns_adj_verb(text, stopword)) for text in train_texts]
        test_texts = [' '.join(nouns_adj_verb(text, stopword)) for text in test_texts]
        fs= "nouns_adj_verbs"

      if stopword == 0:
        st= "оставили"
      else:
        st= "удалили"

      res = train_and_test(train_texts, test_texts, train_labels, test_labels, d, fs, st)
      for i in res:
        itog.append(i)
      print(itog)

df_results = pd.DataFrame(itog, columns=["Dataset", "Vectorizer", "Classifier", "Accuracy", "F1 Score", "Feature space", "Stopwords"])
df_results.to_excel("results.xlsx", index=False)

In [ ]:
itog

In [ ]:
import pandas as pd
df_results = pd.DataFrame(itog, columns=["Dataset", "Vectorizer", "Classifier", "Accuracy", "F1 Score", "Feature space", "Stopwords"])
df_results.to_excel("results.xlsx", index=False)

# LSA через TruncatedSVD

Изменяем параметр k, уменьшая размерность матрицы признаков

In [ ]:
from sklearn.decomposition import TruncatedSVD

def lsa_k(train_text, train_labels, test_text, test_labels, dataset_name, feature_space, stopword):
  results = []

  for k in [100, 500, 1000]:
    for vec_name, vectorizer in vectorizers.items():
      pipeline = Pipeline([
        ("vectorizer", vectorizer),
        ("svd", TruncatedSVD(n_components=k, random_state=42)),
        ("clf", LogisticRegression(max_iter=2000, random_state=42))
        ])

      pipeline.fit(train_text, train_labels)
      y_pred = pipeline.predict(test_text)
      accuracy = accuracy_score(test_labels, y_pred)
      f1 = f1_score(test_labels, y_pred, average="macro")

      results.append([k, dataset_name, vec_name, accuracy, f1, feature_space, stopword])
  return results

In [ ]:
itog_lsa = []
# теперь проводим эксперименты и сохраняем результаты
for d in ['news', 'imdb']:
  for feature_space in range(1, 5):
    for stopword in [0, 1]:
      if d == 'news':
        train_texts = train_texts_news
        test_texts = test_texts_news
        train_labels = train_labels_news
        test_labels = test_labels_news
      else:
        train_texts = train_texts_imdb
        test_texts = test_texts_imdb
        train_labels = train_labels_imdb
        test_labels = test_labels_imdb

      if feature_space == 1:
        train_texts = [' '.join(all_words(text, stopword)) for text in train_texts]
        test_texts = [' '.join(all_words(text, stopword)) for text in test_texts]
        fs= "all_words"
      elif feature_space == 2:
        train_texts = [' '.join(nouns(text, stopword)) for text in train_texts]
        test_texts = [' '.join(nouns(text, stopword)) for text in test_texts]
        fs = "nouns"
      elif feature_space == 3:
        train_texts = [' '.join(nouns_adj(text, stopword)) for text in train_texts]
        test_texts = [' '.join(nouns_adj(text, stopword)) for text in test_texts]
        fs= "nouns_adj"
      else:
        train_texts = [' '.join(nouns_adj_verb(text, stopword)) for text in train_texts]
        test_texts = [' '.join(nouns_adj_verb(text, stopword)) for text in test_texts]
        fs= "nouns_adj_verbs"

      if stopword == 0:
        st= "оставили"
      else:
        st= "удалили"

      res = lsa_k(train_texts, train_labels, test_texts, test_labels, d, fs, st)
      for i in res:
        itog_lsa.append(i)
      print(len(itog_lsa))

df_results = pd.DataFrame(itog_lsa, columns=["k", "Dataset", "Vectorizer", "Accuracy", "F1 Score", "Feature space", "Stopwords"])
df_results.to_excel("results_lsa.xlsx", index=False)